In [1]:
import pygame
import math
import random
pygame.init()

pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


(5, 0)

In [2]:
class DrawInfo:
    black = 0,0,0
    white= 255,255,255
    red=255,0,0
    green = 0, 255, 0
    bgcolor = white
    greys = [ (128,128,128),
             (160,160,160),
             (190,190,190) ]
    side_pad=100 #for both sides
    top_pad= 150 
    
    font = pygame.font.SysFont('comicsans', 20)
    large_font = pygame.font.SysFont('comicsans', 30)
    
    def __init__(self, width, height, lst):
        self.width=width
        self.height=height
        
        self.window = pygame.display.set_mode((width,height))
        pygame.display.set_caption("Sorting Algorithm Visualizer")
        self.set_list(lst)
    
    def set_list(self, lst):
        self.lst = lst
        self.max_val = max(lst)
        self.min_val = min(lst)
        
        self.block_width= round((self.width - self.side_pad ) / len(lst))
        self.block_height = math.floor((self.height- self.top_pad) / (self.max_val - self.min_val ))
        self.start_x =self.side_pad // 2

In [3]:
#generate a list

def generate_list(n,min_val,max_val): #n= arr size
    lst =[]
    for _ in range(n):
        val= random.randint(min_val,max_val)
        lst.append(val)
    return lst


In [4]:
#draw the list
def draw_list(draw_info,color_positions={}, clear_bg=False):
    lst = draw_info.lst
    
    if clear_bg:
        clear_rect = (draw_info.side_pad//2, draw_info.top_pad, 
                        draw_info.width - draw_info.side_pad, draw_info.height - draw_info.top_pad)
        pygame.draw.rect(draw_info.window, draw_info.bgcolor, clear_rect)

    for i, val in enumerate(lst):
        x= draw_info.start_x + i *draw_info.block_width
        y= draw_info.height - (val - draw_info.min_val) * draw_info.block_height
        color = draw_info.greys[i % 3]
        
        
        if i in color_positions:
            color = color_positions[i]
            
        pygame.draw.rect(draw_info.window, color, (x,y,draw_info.block_width,draw_info.height))
        
    if clear_bg:
        pygame.display.update()

In [5]:
#draw the screen
def draw(draw_info, algo_name, ascending):
    draw_info.window.fill(draw_info.bgcolor)
    title = draw_info.large_font.render(f"{algo_name} - {'Ascending' if ascending else 'Descending'}", 1, draw_info.green)
    draw_info.window.blit(title, (draw_info.width/2 - title.get_width()/2 , 8))

    controls= draw_info.font.render("PRESS: SPACE = Start Sorting | R = Reset | A = Ascending | D = Descending",
                                    1, draw_info.black)
    draw_info.window.blit(controls , (draw_info.width/2 - controls.get_width()/2 ,45))
    draw_list(draw_info)
    pygame.display.update()

In [6]:
def insertin_sort(draw_info , ascending = True):
    lst = draw_info.lst
    for i in range(1, len(lst)):
        current = lst[i]

        while True:
            ascending_sort = i > 0 and lst[i - 1] > current and ascending
            descending_sort = i > 0 and lst[i - 1] < current and not ascending

            if not ascending_sort and not descending_sort:
                break

            lst[i] = lst[i - 1]
            i = i - 1
            lst[i] = current
            draw_list(draw_info, {i - 1: draw_info.green, i: draw_info.red}, True) #red is being sorted, green is comparing
            yield True

    return lst


In [7]:
def main():
    run = True
    clock = pygame.time.Clock()
    sorting = False
    ascending = True
    
    sorting_algorithm = insertin_sort
    sorting_algo_name = "Insertion Sort"
    sorting_algorithm_generator = None
    
    #generate_list vars
    n= 50
    min_val=0
    max_val=100
    
    
    lst= generate_list(n, min_val, max_val)
    draw_info = DrawInfo(800, 600,lst)#width, height, lst
    
    while run:
        clock.tick(10)
        
        if sorting:
            try:
                next(sorting_algorithm_generator)
            except StopIteration:
                sorting = False
        else:
            draw(draw_info, sorting_algo_name, ascending)
            
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT: #for the X button
                run = False
            
            if event.type != pygame.KEYDOWN:
                continue
            
            if event.key == pygame.K_r:
                lst= generate_list(n, min_val, max_val)
                draw_info.set_list(lst)
                sorting = False
            elif event.key == pygame.K_SPACE and sorting== False:
                sorting = True
                sorting_algorithm_generator = sorting_algorithm(draw_info, ascending)
            elif event.key == pygame.K_a and not sorting :
                ascending = True
            elif event.key == pygame.K_d and not sorting :
                ascending = False
                
    pygame.quit()
    
    
if __name__ == "__main__":
    main()